<a href="https://colab.research.google.com/github/jhackisneros/VISION_ARTIFICIAL/blob/main/07_hyperparam_search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# VISION_ARTIFICIAL - Notebook 07: Búsqueda de hiperparámetros

🔹 Importar librerías

In [ ]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import ParameterGrid
import json, os

🔹 Cargar datos procesados

In [ ]:
processed_dir = "../data/processed/"
x_train = np.load(processed_dir + "x_train.npy")
y_train = np.load(processed_dir + "y_train.npy")
x_test  = np.load(processed_dir + "x_test.npy")
y_test  = np.load(processed_dir + "y_test.npy")


FileNotFoundError: [Errno 2] No such file or directory: '../data/processed/x_train.npy'

🔹 Definir rango de hiperparámetros

In [ ]:
param_grid = {
    'filters1': [32, 64],
    'filters2': [64, 128],
    'dense_units': [64, 128],
    'dropout_rate': [0.0, 0.3],
    'optimizer': ['adam', 'rmsprop'],
    'batch_size': [32, 64]
}


🔹 Carpeta para guardar resultados

In [ ]:
os.makedirs("../experiments", exist_ok=True)

🔹 Grid search manual

In [ ]:
results = []
for params in ParameterGrid(param_grid):
    print("Entrenando con parámetros:", params)

    model = Sequential([
        Conv2D(params['filters1'], (3,3), activation='relu', padding='same', input_shape=(32,32,3)),
        MaxPooling2D((2,2)),
        Conv2D(params['filters2'], (3,3), activation='relu', padding='same'),
        MaxPooling2D((2,2)),
        Flatten(),
        Dense(params['dense_units'], activation='relu'),
        Dropout(params['dropout_rate']),
        Dense(10, activation='softmax')
    ])

    model.compile(optimizer=params['optimizer'], loss='categorical_crossentropy', metrics=['accuracy'])

    history = model.fit(x_train, y_train, validation_data=(x_test, y_test),
                        epochs=10, batch_size=params['batch_size'], verbose=0)

    best_val_acc = max(history.history['val_accuracy'])
    results.append({**params, 'val_accuracy': float(best_val_acc)})
    print("→ Mejor val_accuracy:", best_val_acc)

🔹 Guardar resultados

In [ ]:
with open("../experiments/hyperparam_trials.json", "w") as f:
    json.dump(results, f, indent=4)

print("\n✅ Búsqueda de hiperparámetros completada. Resultados guardados en 'experiments/hyperparam_trials.json'")